In [1]:
import warnings
warnings.simplefilter('ignore')

## 0. Install and import dependencies

In [9]:
# if already installed do not run ! :
#!pip install pystan prophet plotly Cython

In [14]:
import pandas as pd
from prophet import Prophet 
%matplotlib inline

## 1. Read in Data and process dates

In [15]:
df = pd.read_csv('dataset.csv')

## 2. Train Model

## 3. Forecast Away